In [27]:
import numpy as np
import pandas as pd
import math

In [171]:
train = pd.read_csv('watermelon.csv')

In [172]:
test = train.iloc[-1]
train = train.drop(17 , axis=0)

In [173]:
test[0]

'qing'

In [174]:
len(train.loc[train[cols[0]] == 'qing'])

6

In [175]:
m = len(train) #样本数量
yes_melon = train['good'].value_counts().y
no_melon = train['good'].value_counts().n

#计算先验概率 未使用laplace平滑处理 进行先验概率的计算
def p_c():
    p_y = yes_melon / m
    p_n = no_melon / m
    
    return p_y , p_n

p_y , p_n = p_c()

cols = train.columns

In [176]:
m_qing_yes = len( train.loc[(train['color'] == test[0]) & (train['good'] == 'y')] )

In [177]:
def std_norm(x , mean , std):
    a = 1.0 / (math.sqrt(2*math.pi) * std)
    b = -1.0*(x-mean)*(x-mean) / (2*std*std)
    c = math.exp(b)
    d = a*c
    
    return d

def conditional_prob(test):
    size = len(test)
    
    positive = 1.0
    negtive = 1.0
    
    for i in range(size-3):
        pos_number = len( train.loc[(train[cols[i]] == test[i]) & (train['good'] == 'y')])
        positive *= (pos_number/yes_melon)
        #print('positive:' , (pos_number , yes_melon))
        neg_number = len( train.loc[(train[cols[i]] == test[i]) & (train['good'] == 'n')])
        negtive *= (neg_number/no_melon)
        #print('negtive:' , (neg_number,no_melon))

    a = np.array(train.loc[train['good'] == 'y', 'den'])
    positive *= std_norm(test[6] , np.mean(a) , np.std(a))
    #print('positive:' , std_norm(test[6] , np.mean(a) , np.std(a)))

    
    a = np.array(train.loc[train['good'] == 'n', 'den'])
    negtive *= std_norm(test[6] , np.mean(a) , np.std(a))
    #print('negtive:' , std_norm(test[6] , np.mean(a) , np.std(a)))

    b = np.array(train.loc[train['good'] == 'y', 'sugar'])
    positive *= std_norm(test[7] , np.mean(b) , np.std(b))
    #print('positive:' , std_norm(test[7] , np.mean(b) , np.std(b)))

    
    b = np.array(train.loc[train['good'] == 'n', 'sugar'])
    negtive *= std_norm(test[7] , np.mean(b) , np.std(b))
    #print('negtive:' , std_norm(test[7] , np.mean(b) , np.std(b)))

    
    return positive , negtive

In [178]:
positive , negtive = conditional_prob(test)

positive *= p_y
negtive *= p_n

In [179]:
if positive>negtive :
    print('good melon')
else:
    print('bad melon')

good melon


In [180]:
#===========
#===========
#上下 分别是 未使用平滑处理 和 使用平滑处理

In [185]:
#laplce处理
def p_c_laplace():
    p_y = (yes_melon+1) / (m + 2) #因为是2分类问题 所以+2 如果为k分类问题 就+k
    p_n = (no_melon+1) / (m+2)
    
    return p_y , p_n

#laplace平滑处理
def conditional_prob_laplace(test):
    size = len(test)
    
    positive = 1.0
    negtive = 1.0
    
    for i in range(size-3):
        pos_number = len( train.loc[(train[cols[i]] == test[i]) & (train['good'] == 'y')])
        positive *= ( (pos_number + 1)/(yes_melon + 2) )#加2是因为这是一个2分类问题 只有好瓜 坏瓜之分 如果为k分类 就是加k 下面雷同
        #print('positive:' , (pos_number , yes_melon))
        neg_number = len( train.loc[(train[cols[i]] == test[i]) & (train['good'] == 'n')])
        negtive *= ( (neg_number+1) /(no_melon+2) )
        #print('negtive:' , (neg_number,no_melon))

    a = np.array(train.loc[train['good'] == 'y', 'den'])
    positive *= std_norm(test[6] , np.mean(a) , np.std(a))
    #print('positive:' , std_norm(test[6] , np.mean(a) , np.std(a)))

    
    a = np.array(train.loc[train['good'] == 'n', 'den'])
    negtive *= std_norm(test[6] , np.mean(a) , np.std(a))
    #print('negtive:' , std_norm(test[6] , np.mean(a) , np.std(a)))

    b = np.array(train.loc[train['good'] == 'y', 'sugar'])
    positive *= std_norm(test[7] , np.mean(b) , np.std(b))
    #print('positive:' , std_norm(test[7] , np.mean(b) , np.std(b)))

    
    b = np.array(train.loc[train['good'] == 'n', 'sugar'])
    negtive *= std_norm(test[7] , np.mean(b) , np.std(b))
    #print('negtive:' , std_norm(test[7] , np.mean(b) , np.std(b)))

    return positive , negtive

In [186]:
laplace_test = test

In [187]:
laplace_test['qi'] = 'qing' #敲声清脆这个属性 在训练样本中没有出现 这时候 就需要laplace平滑处理

In [188]:
positive , negtive = conditional_prob(laplace_test)
(positive , negtive) #0.0 0.0 就是因为没有平滑处理 导致分子位置为0 相乘结果为0

(0.0, 0.0)

In [189]:
positive , negtive = conditional_prob_laplace(laplace_test)
p_y , p_n = p_c_laplace()
positive *= p_y
negtive *= p_n

In [190]:
(positive , negtive)

(0.0058656200106201907, 2.5317380026971431e-05)